In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from autoencoder import Autoencoder

In [ ]:
def load_data(data_dir, image_size=(64, 64, 3)):
    image_names = os.listdir(data_dir)
    image_names = [x for x in image_names if x.endswith('.jpg') or x.endswith('.png') or x.endswith('.jpeg')]
    train_data = []
    for image_name in image_names:
        image = cv2.imread(os.path.join(data_dir, image_name))
        image = image / 255.0
        train_data.append(image)
    return train_data

In [ ]:
def dssim_loss(y_true, y_pred):
    return 1/2 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))/2

def model_name(dataset_name, latent_dim, training_loss, batch_size):
    return f'{dataset_name}_dim_{latent_dim}_loss_{training_loss}_batch_{batch_size}.hdf5'

In [ ]:
def train_model(input_shape=(64, 64, 3), latent_dim=50, training_loss='ssim', batch_size=8, epochs=20):

    # Load data
    X_train = load_data("images/", input_shape)
    X_train = np.array(X_train)

    Y_train = X_train
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)
    print(f'Loaded data: train {X_train.shape}, validation {Y_val.shape}')

    # Create autoencoder
    autoencoder = Autoencoder(input_shape=input_shape, latent_dim=latent_dim)

    # Set training loss function and optimizer
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=10000,
        decay_rate=0.9)

    opt = Adam(learning_rate=lr_schedule)

    if training_loss == 'mse':
        autoencoder.compile(loss='mse', optimizer=opt)
    elif training_loss == 'ssim':
        autoencoder.compile(loss=dssim_loss, optimizer=opt)

    # Training
    autoencoder.fit(
      x=X_train, y=Y_train,
      epochs=epochs, batch_size=batch_size,
      shuffle=True, validation_data=(X_val, Y_val),
      callbacks=[])
    
    return autoencoder


In [ ]:
dataset_name = 'grid'
latent_dim = 30
batch_size = 8
training_loss = 'ssim'

autoencoder = train_model( 
    input_shape=(64, 64, 3),
    latent_dim = latent_dim,
    training_loss = training_loss,
    batch_size = batch_size,
    epochs = 80)

autoencoder.encoder.save_weights("models/encoder30.h5")